# Prepare

In [ ]:
!pip install --quiet transformers
!pip install --quiet sentencepiece
!pip install --quiet pytorch-lightning

     |████████████████████████████████| 3.1 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 51.1 MB/s 
     |████████████████████████████████| 3.3 MB 44.7 MB/s 
     |████████████████████████████████| 59 kB 6.1 MB/s 
     |████████████████████████████████| 895 kB 35.7 MB/s 
     |████████████████████████████████| 1.2 MB 4.5 MB/s 
     |████████████████████████████████| 523 kB 5.2 MB/s 
     |████████████████████████████████| 132 kB 51.6 MB/s 
     |████████████████████████████████| 329 kB 47.8 MB/s 
     |████████████████████████████████| 829 kB 42.3 MB/s 
     |████████████████████████████████| 1.1 MB 36.2 MB/s 
     |████████████████████████████████| 271 kB 32.8 MB/s 
     |████████████████████████████████| 192 kB 48.1 MB/s 
     |████████████████████████████████| 160 kB 48.1 MB/s 


In [ ]:
import pandas as pd
import numpy as np


from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping


import pytorch_lightning as pl

Unzipping...

/usr/local/lib/python3.7/dist-packages/google_drive_downloader/google_drive_downloader.py:78: UserWarning: Ignoring `unzip` since "1HGDbDVoQXL50lJrrc4K77AOwmOF2b-Cc" does not look like a valid zip file
  warnings.warn('Ignoring `unzip` since "{}" does not look like a valid zip file'.format(file_id))


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import json
file = open("data.json", encoding="utf8")
file = json.load(file)

In [ ]:
data = []
for block in file:
  if len(block['Categories']) > 0:
    for i in range(len(block['Categories'])):
      data.append((block['Categories'][i]['Sentence'], block['Categories'][i]['Phrase'], block['Comment']))

In [ ]:
df = pd.DataFrame(data, columns = ['sentence', 'category','comment'])

In [ ]:
import nltk
from nltk import word_tokenize, sent_tokenize
# from PersianStemmer import PersianStemmer
from nltk.stem import WordNetLemmatizer
import string

import codecs,re
def arabic_to_farsi(text):
    text = re.sub(r'[كﮑﮐﮏﮎﻜﻛﻚﻙ]', r'ک', text)
    text = re.sub(r'[ىىىﻴﻢﻳﻲﻱﻰىىﻯي]', r'ی', text)
    return text

def cleaning(text):
  text = str(text)
  text = re.sub(r'#', ' ', text)
  text = re.sub(r'_', ' ', text)
  text = text.replace('\u200c', ' ')
  text = re.sub(r'([ا-ی])\1{2,}', r'\1', text) # bbb+ -> b at least 2 char
  text = re.sub(r'[^\w\s]',' ',text)
  text = re.sub(r'\@\w*', ' ', text)
  text = re.sub(r'\d+', ' ', text)
  text = re.sub(r'$', ' ', text)
  text = re.sub(r"[a-zA-Z0-9]+", " ",text)
  text = re.sub(r'<(.*?)>', ' ', text)
  text = re.sub(r'https?:\/\/.*[\r\n]*', ' ', text)
  text=re.sub(r'[۰۱۲۳۴۵۶۷۸۹٠١١٢٣٤٥٦٧٧٨٨٩٩●]',' ',text)
  text=re.sub(r'[\·\♦\٭\\,\^\|\˝\٬\’\”\‹\▪\○¼ç½éêüəıœ™Ááàäāćíłñū©ٰٔ]',' ',text)
  text=re.sub(r'[ًٌٍَُِّْ]',' ',text)
  text=re.sub(r'[]',r' ',text)
  text=re.sub(r'[\–\—…°≈≠±≤≥\−×÷√٪→←↔↑↓\#\٫]',u' ',text) 
  text=text.replace(u'•',u' ').replace(u'ˈ',u' ').replace(u'؛',u' ').replace(u'/',u' ').replace(u'ۀ',u'هٔ').replace(u"﴿",u' ').replace(u"﴾",u' ').replace(u"'",u' ').replace(u'\\',u' ').replace(u'[',u' ').replace(u']',u' ').replace(u'?',u' ').replace(u'؟',u' ').replace(u')',u' ').replace(u'_',u' ').replace(u'(u',u' ').replace(u'}',u' ').replace(u'{',u' ').replace(u'.',u' ').replace(u'>',u' ').replace(u'<',u' ')
  text=text.replace(u'`',u' ').replace(u'\t',u' ').replace(u'=',u' ').replace(u'»',u' ').replace(u'«',u' ').replace(u'~',u' ').replace(u'!',u' ').replace(u'@',u' ').replace(u'$',u' ').replace(u',u',u' ').replace(u'%',u' ').replace(u'،',u' ').replace(u'-',u' ').replace(u';',u' ').replace(u':',u' ').replace(u'*',u' ').replace(u'"',u' ').replace(u'&',u' ').replace(u'#',u' ').replace(u'+',u' ')
  text=re.sub(r'[\n\r]{2,}',u'\n',text)
  text = re.sub(u'(\u202A|\u202B|\u202C|\u202D|\u202E|\u200F|\uFEFF|\u2003|\¬|\­)',u'\u200C', text)#حذف کارکترهای تغییرجهت
  text = re.sub(u'‌{2,}', u'‌', text) # پشت‌سرهم
  text = re.sub(u'‌(?![ئاآأإژزرذدوؤةبپتثجچحخسشصضطظعغفقکگلمنهیيًٌٍَُِّْٰٓٔ]|[\u0900-\u097F]|ֹ)', u'', text) # در پس
  text = re.sub(u'(?<![ئبپتثجچحخسشصضطظعغفقکگلمنهیيًٌٍَُِّْٰٓٔ]|[\u0900-\u097F]|f|ֹ)‌', u'', text) # در پیش
  text=text.replace(u'­',u' ').replace(u'­',u' ').replace(u'ـ',u' ').replace(u'ـ',u' ').replace(u'ـ',u' ').replace(u'ـ',u' ').replace(u'',u' ')
  text=text.replace(u'',u' ')
  text = re.sub(u'‌{2,}', u'‌', text) # پشت‌سرهم
  text = re.sub(u'(\u00A0)',u' ', text).replace(u'(',u' ').replace(u')',u' ')
  text=text.replace(u'    ',u' ').replace(u'    ',u' ').replace(u'   ',u' ').replace(u'  ',u' ').replace(u'  ',u' ').replace(u'  ',u' ')
  # text=arabic_to_farsi(text)
  return text.strip()

In [ ]:
df['cleaned_sentence'] = df.sentence.apply(cleaning)
df['cleaned_comment'] = df.comment.apply(cleaning)

In [ ]:
for i in range(len(df['category'])):
  if df['category'][i] == None:
    df['category'][i] = 'ندارد'

# model

In [ ]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, AdamW
model_size = "small"
model_name = f"persiannlp/mt5-{model_size}-parsinlu-squad-reading-comprehension"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)


batch_size = 8
max_len = 80
RANDOM_SEED = 42

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/383 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/609 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [ ]:
from sklearn.model_selection import train_test_split

train_df,test_df = train_test_split(df, test_size=0.1, random_state=RANDOM_SEED,shuffle=True)
train_df,valid_df = train_test_split(train_df, test_size=0.1, random_state=RANDOM_SEED,shuffle=True)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
train_df.shape, valid_df.shape, test_df.shape

((4368, 5), (486, 5), (540, 5))

In [ ]:
class Comment(Dataset):

  def __init__(
    self, 
    data: pd.DataFrame, 
    tokenizer: tokenizer, 
    source_max_token_len: int = 130,
    target_max_token_len: int = 80
  ):
    self.tokenizer = tokenizer
    self.data = data
    self.source_max_token_len = source_max_token_len
    self.target_max_token_len = target_max_token_len
    
  def __len__(self):
    return len(self.data)

  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]

    source_encoding = self.tokenizer.encode_plus(
      data_row['category'],
      data_row['cleaned_comment'],
      add_special_tokens=True,
      max_length=self.source_max_token_len,
      padding="max_length",
      truncation='only_second',
      return_attention_mask=True,
      return_tensors='pt',
    )
    target_encoding = self.tokenizer.encode_plus(
      data_row['cleaned_sentence'],
      add_special_tokens=True,
      max_length=self.target_max_token_len,
      padding="max_length",
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt',
    )
    
    labels = target_encoding["input_ids"]
    labels[labels == 0] = -100

    return dict(
      question=data_row['category'],
      context=data_row['cleaned_comment'],
      answer_text=data_row['cleaned_sentence'],
      input_ids=source_encoding["input_ids"].flatten(),
      attention_mask=source_encoding["attention_mask"].flatten(),
      labels=labels.flatten()
    )

In [ ]:
train_dataset = Comment(train_df, tokenizer)
validate_dataset = Comment(valid_df, tokenizer)
test_dataset = Comment(test_df, tokenizer)


In [ ]:
class CommentDataModule(pl.LightningDataModule):

  def __init__(self, train_df, test_df, tokenizer, batch_size=8, source_max_token_len: int = 130, target_max_token_len: int = 80):
    super().__init__()
    self.batch_size = batch_size
    self.train_df = train_df
    self.test_df = test_df
    self.tokenizer = tokenizer
    self.data = data
    self.source_max_token_len = source_max_token_len
    self.target_max_token_len = target_max_token_len

  def setup(self, stage=None):
    self.train_dataset = Comment(
      self.train_df,
      self.tokenizer,
      self.source_max_token_len,
      self.target_max_token_len,
    )

    self.test_dataset = Comment(
      self.test_df,
      self.tokenizer,
      self.source_max_token_len,
      self.target_max_token_len,
    )

  def train_dataloader(self):
    return DataLoader(
      self.train_dataset,
      batch_size=self.batch_size,
      shuffle=True,
      num_workers=2
    )

  def val_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=1,
      num_workers=2
    )

  def test_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=1,
      num_workers=2
    )

In [ ]:
data_module = CommentDataModule(
  train_df,
  valid_df,
  tokenizer,
  batch_size=batch_size,
)

In [ ]:
data_module.train_df[data_module.train_df['category'] != None]

,sentence,category,comment,cleaned_sentence,cleaned_comment
0,بسته رو خیلی سریع ارسال کردن.,ارسال,بسته رو خیلی سریع ارسال کردن. پشتیبانیشونم خیل...,بسته رو خیلی سریع ارسال کردن,بسته رو خیلی سریع ارسال کردن پشتیبانیشونم خیلی...
1,این فروشکاه رو دوست دارم قیمت هاش عالیه .,قیمت,این فروشکاه رو دوست دارم قیمت هاش عالیه,این فروشکاه رو دوست دارم قیمت هاش عالیه,این فروشکاه رو دوست دارم قیمت هاش عالیه
2,خرید خوب و ارسال به موقع.,عمومی,خرید خوب و ارسال به موقع.,خرید خوب و ارسال به موقع,خرید خوب و ارسال به موقع
3,میشه لطفا دایرکت رو چک کنین ، امروز سفارشاتم ر...,پشتیبانی,سلام ... من پولمو کامل واریز کردم سفارشمم کامل...,میشه لطفا دایرکت رو چک کنین امروز سفارشاتم رو ...,سلام من پولمو کامل واریز کردم سفارشمم کامل گفت...
4,با سلام و خسته نباشید سپاسگذار از برخورد محترم...,خدمات,با سلام و خسته نباشید سپاسگذار از برخورد محترم...,با سلام و خسته نباشید سپاسگذار از برخورد محترم...,با سلام و خسته نباشید سپاسگذار از برخورد محترم...
...,...,...,...,...,...
4363,تمام اجناس به قیمت خوب و اصل و پلمپ شده بودند ...,کارایی آسان,به نظر من فروشگاه خوبی است، از این سایت یک سی...,تمام اجناس به قیمت خوب و اصل و پلمپ شده بودند ...,به نظر من فروشگاه خوبی است از این سایت یک سی پ...
4364,سلام دوستان ما اینترنتی خرید کردیم.,ندارد,سلام دوستان ما اینترنتی خرید کردیم و بسیار گوش...,سلام دوستان ما اینترنتی خرید کردیم,سلام دوستان ما اینترنتی خرید کردیم و بسیار گوش...
4365,لباس‌های بی‌نهایت با کیفیت.,کیفیت کالا,لباس‌های بی‌نهایت با کیفیت,لباس های بی نهایت با کیفیت,لباس های بی نهایت با کیفیت
4366,من قصد خرید مانیتور برای طراحی داشتم و یک مانی...,پشتیبانی,من قصد خرید مانیتور برای طراحی داشتم و یک مانی...,من قصد خرید مانیتور برای طراحی داشتم و یک مانی...,من قصد خرید مانیتور برای طراحی داشتم و یک مانی...


In [ ]:
class ReadingComprehension(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = model = MT5ForConditionalGeneration.from_pretrained(model_name)

  def forward(self, input_ids, attention_mask, labels=None):
    output = self.model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        labels=labels
    )

    return output.loss, output.logits

  def training_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return loss

  def validation_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss

  def test_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("test_loss", loss, prog_bar=True, logger=True)
    return loss

  def configure_optimizers(self):
    return AdamW(self.parameters(), lr = 1e-5)

In [ ]:
model = ReadingComprehension()

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints',
    filename='best-checkpoint',
    save_top_k=1,
    verbose=True,
    monitor='val_loss',
    mode='min'
)

In [ ]:
trainer = pl.Trainer(
    checkpoint_callback=checkpoint_callback,
    max_epochs=2,
    gpus=1,
    progress_bar_refresh_rate=10
)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:148: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7ff319c24c50>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7ff319c24c50>)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=10)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Tr

In [ ]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                        | Params
------------------------------------------------------
0 | model | MT5ForConditionalGeneration | 300 M 
------------------------------------------------------
300 M     Trainable params
0         Non-trainable params
300 M     Total params
1,200.707 Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/data.py:60: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 5. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/data.py:60: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 8. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/data.py:60: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 11. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/data.py:60: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 10. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/data.py:60: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 4. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguo

Validating: 0it [00:00, ?it/s]

In [ ]:
def run_model(paragraph, question, **generator_args):
    input_ids = tokenizer.encode(question + "\n" + paragraph, return_tensors="pt")
    res = model.model.generate(input_ids, **generator_args)
    output = tokenizer.batch_decode(res, skip_special_tokens=True)
    print(output)
    return output

In [ ]:
sentences, comments, categories = test_df['cleaned_sentence'], test_df['cleaned_comment'], test_df['category']
generated = []
for i in range(len(sentences)):
  generated += run_model(comments[i], categories[i])

['تماس با پشتیبانی گرفتم و از گارانتی و شرایط پرسیدم و']
['وقت مارکت تونست مرخصی هم برام بگیره بفرمایید تا']
['قیمت هم پشتیبانی من یه لپتاپ ایسوس خریدم ازشون']
['بهترین تجربه خرید اینترنتی بین هزاران خرید اینترنتی که در']
['قیمت عالی و پشتیبانی بسیار سریع و خوش برخورد ممنونم']
['تماس گرفتن باهام که امروز میخوشون هیچ جایی پیدا نمی']
['تماس ارسال مادر بورد را با بهترین قیمت یک روزه بدستم رسید']
['از ایسل مزه یک خرید خوب اینترنتی را بعد از مدتها چشید']
['قیمت های دلار و بالاتر']
['برای خرید زنگ زدم بهشون برای ایرپاد پرو رنگی خیلی']
['ارسال بهترین و سریع ترین خرید اینترنتی بود که داشتم ساعت']
['یکی از بهترین فروشگاه های اینترنتی در ایران می باشد دارای سرویس']
['اگر نمیخواهید بیایید زنگ بزنید و گفتوئید مردم مسخر']
['قیمت مناسب تر تر']
['در موقع و منظم سرتیام سرتایآممم فرستادن']
['جلودار دوره']
['دقیقا همون کالای اصلی که ارائه داده بودن را ارسال کردن']
['فروشگاه قیمتهای مناسب و بروز و ارسال بموقع']
['شبکه']
['منصفانه هست من یک آسیاب دلونگی که همه جه بالاتر از']
['فروشگاه خریداری شده حتما بدس

KeyboardInterrupt: ignored